In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")
import re
import nltk
from nltk.corpus import stopwords

In [6]:
data = pd.read_csv('data/tags.csv')

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(encoding='utf-8',stop_words=stopwords.words("russian"))

In [60]:
vectors = cv.fit_transform(data['tags'].values.astype('U')).toarray()

In [61]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [62]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [63]:
cv.get_feature_names()

['000',
 '000экземпляр',
 '006',
 '10',
 '100',
 '1000',
 '100000',
 '1001',
 '100ть',
 '102',
 '103',
 '1048',
 '105251танк',
 '108',
 '108ть',
 '1099',
 '10быть',
 '10го',
 '10идеальный',
 '10й',
 '10летие',
 '10летний',
 '10я',
 '11',
 '110',
 '112263',
 '1127',
 '1137',
 '114',
 '1147',
 '1148',
 '1150',
 '116',
 '1164',
 '117',
 '1185',
 '11го',
 '11й',
 '11летний',
 '11я',
 '12',
 '120',
 '121',
 '122',
 '1235',
 '125',
 '127',
 '1270',
 '12й',
 '12летний',
 '12я',
 '13',
 '130',
 '1300',
 '1307',
 '1314',
 '133',
 '1346',
 '1348',
 '1356',
 '1395',
 '13летний',
 '14',
 '140',
 '1408пять',
 '1409',
 '1453',
 '1470ха',
 '1471',
 '1499',
 '14летний',
 '15',
 '150',
 '1500',
 '1500000',
 '1511',
 '1517',
 '1520',
 '152мма',
 '1537',
 '1541',
 '1542',
 '155',
 '1558',
 '1580ха',
 '1590',
 '15е',
 '15й',
 '15летний',
 '16',
 '160',
 '1600',
 '1606',
 '160летний',
 '1612',
 '1617',
 '1617м',
 '1620',
 '1626',
 '1630',
 '165',
 '1652',
 '1666',
 '16671745',
 '1692',
 '1699',
 '16й',
 '1

In [64]:
len(cv.get_feature_names())

45636

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
similarity = cosine_similarity(vectors)

In [67]:
similarity

array([[1.        , 0.235725  , 0.14896906, ..., 0.        , 0.        ,
        0.        ],
       [0.235725  , 1.        , 0.13054598, ..., 0.        , 0.        ,
        0.        ],
       [0.14896906, 0.13054598, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [68]:
similarity[0]

array([1.        , 0.235725  , 0.14896906, ..., 0.        , 0.        ,
       0.        ])

In [69]:
type(similarity)

numpy.ndarray

In [70]:
list(zip(list(range(len(similarity[0].tolist()))),similarity[0].tolist()))

TypeError: 'Int64Index' object is not callable

In [72]:
a = [1,2,3,4,5,6,7,8,9]
a

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [73]:
enumerate(a)

In [74]:
list(enumerate(a))

TypeError: 'Int64Index' object is not callable

In [ ]:
list(enumerate(similarity[0]))

In [ ]:
sorted(similarity[0],reverse=True)

In [ ]:
opop = data[data['name'] == 'Благословение небожителей. Том 2 '].index[0]

In [ ]:
def recommend(rusbook):
    rusbook_index = str(data[data['name'] == rusbook].index[0])
    distances = similarity[rusbook_index]
    rusbook_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in rusbook_list:
        print(data.iloc[i[0]].name)

In [ ]:
recommend('Благословение небожителей. Том 2 ')

In [ ]:
import pickle

In [ ]:
pickle.dump(data.to_dict(),open('books_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))